In [1]:
import numpy as np
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential, load_model
from keras.utils import to_categorical
from sklearn.datasets import fetch_olivetti_faces
from sklearn.metrics import (accuracy_score, f1_score, precision_score,
                             recall_score)
from sklearn.model_selection import train_test_split

2024-01-22 21:36:25.598635: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Reconhecimento de Imagem

### Carregamento dos dados

In [2]:
# Carregar o dataset
dataset = fetch_olivetti_faces(shuffle=True, random_state=42)

# Dividir o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(dataset.images, dataset.target, test_size=0.2,
                                                    random_state=42)

# Converter os dados para o formato correto
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Definir variáveis globais
INPUT_SHAPE = (X_train.shape[1], X_train.shape[2], 1, )
NUM_CLASSES = y_train.shape[1]
BATCH_SIZE = 32

print(f"input_shape: {INPUT_SHAPE}\n" \
      f"num_classes: {NUM_CLASSES}\n")

print(f"X_train.shape: {X_train.shape},\t" \
      f"X_test.shape: {X_test.shape}\n" \
      f"y_train.shape: {y_train.shape},\t" \
      f"y_test.shape: {y_test.shape}")

input_shape: (64, 64, 1)
num_classes: 40

X_train.shape: (320, 64, 64),	X_test.shape: (80, 64, 64)
y_train.shape: (320, 40),	y_test.shape: (80, 40)


### Treinamento do modelo

In [3]:
# Definindo o modelo
def build_model():
    model = Sequential()
    model.add(Conv2D(30, kernel_size = (3, 3), activation='relu', input_shape=INPUT_SHAPE))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    
    model.compile(
        loss = 'categorical_crossentropy',
        optimizer = 'adam',
        metrics = ["accuracy"]
    )
    
    return model

# Treinando o modelo
try :
    model = load_model("../models/model.h5")
except:
    model = build_model()
    history = model.fit(X_train, y_train, epochs=25, batch_size=BATCH_SIZE)
    model.save("../models/model.h5")

Epoch 1/25


2024-01-22 21:36:27.071244: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


10/10 [==============================] - 1s 37ms/step - loss: 4.0423 - accuracy: 0.0344
Epoch 2/25
10/10 [==============================] - 0s 32ms/step - loss: 3.6713 - accuracy: 0.0938
Epoch 3/25
10/10 [==============================] - 0s 31ms/step - loss: 3.1396 - accuracy: 0.3000
Epoch 4/25
10/10 [==============================] - 0s 32ms/step - loss: 2.6087 - accuracy: 0.6500
Epoch 5/25
10/10 [==============================] - 0s 38ms/step - loss: 1.9403 - accuracy: 0.8594
Epoch 6/25
10/10 [==============================] - 0s 45ms/step - loss: 1.2513 - accuracy: 0.9406
Epoch 7/25
10/10 [==============================] - 0s 48ms/step - loss: 0.7979 - accuracy: 0.9406
Epoch 8/25
10/10 [==============================] - 0s 40ms/step - loss: 0.5349 - accuracy: 0.9656
Epoch 9/25
10/10 [==============================] - 0s 39ms/step - loss: 0.3615 - accuracy: 0.9719
Epoch 10/25
10/10 [==============================] - 0s 41ms/step - loss: 0.2353 - accuracy: 0.9844
Epoch 11/25
10/10 [=

### Avaliar o modelo

In [4]:
predict = np.argmax(model.predict(X_test), axis=1)
y_test = np.argmax(y_test, axis=1)

print(f"Accuracy: {accuracy_score(y_test, predict)}\n" \
        f"Precision: {precision_score(y_test, predict, average='macro')}\n" \
        f"Recall: {recall_score(y_test, predict, average='macro')}\n" \
        f"F1: {f1_score(y_test, predict, average='macro')}\n")


3/3 [==============================] - 0s 3ms/step
Accuracy: 0.9375
Precision: 0.9704761904761904
Recall: 0.95
F1: 0.9493424036281177

